In [18]:
from __future__ import print_function

import os
import numpy as np
import cv2

from PIL import Image
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

import torch
import torch.backends.cudnn as cudnn
from torchvision.transforms import Compose, CenterCrop, ToTensor, Resize

In [19]:
# ===========================================================
# input image setting
# ===========================================================
GPU_IN_USE = torch.cuda.is_available()
if GPU_IN_USE:
    cudnn.benchmark = True
device = torch.device('cuda' if GPU_IN_USE else 'cpu')

In [20]:
# ===========================================================
# model import & setting
# ===========================================================
print('avaliable model:')
print('-'*10)
filepath='/home/guozy/BISHE/SR_Collection/result/'
subpath = os.listdir(filepath)
subpath.sort()
count = -1
for p in subpath:
    count += 1
    print('{},{}'.format(count, p))
print('-' * 10)

choice=int(input('input the number of the determined model\n'))
filepath=filepath+subpath[choice]+'/model.pth'

print('load model')
model = torch.load(filepath)
model = model.to(device)

avaliable model:
----------
0,2023-03-06_22:52:43
1,2023-03-06_23:05:21
2,2023-03-06_23:07:00
3,SRCNN_pretrained
----------
load model


In [21]:
# ===========================================================
# compare with origin to upsample
# ===========================================================

img = Image.open('/home/guozy/BISHE/SR_Collection/dataset/BSDS300/images/test/69015.jpg')
data = (ToTensor()(img))
data = data.to(device)
out = model(data)
out = out.permute(1,2,0).cpu()
out = out.detach().numpy()
out*= 255.0
out = out.clip(0, 255).astype(np.int8)
out = Image.fromarray(out, mode="RGB")
out.save('/home/guozy/BISHE/SR_Collection/result/'+ subpath[choice]+'/origin_to_upsample_by_NN.jpg')


img=cv2.imread("/home/guozy/BISHE/SR_Collection/dataset/BSDS300/images/test/69015.jpg")
w=img.shape[1]  
h=img.shape[0]
img=cv2.resize(img,(w-1,h-1),interpolation=cv2.INTER_CUBIC)
upsample=cv2.resize(img,(w*2,h*2),interpolation=cv2.INTER_CUBIC)
cv2.imwrite('/home/guozy/BISHE/SR_Collection/result/'+ subpath[choice]+'/origin_to_upsample_by_Bicubic.jpg',upsample,[int(cv2.IMWRITE_JPEG_QUALITY), 100])

True

In [22]:
# ===========================================================
# compare with downsample to origin
# ===========================================================
downsample=cv2.resize(img,(w//2,h//2),interpolation=cv2.INTER_CUBIC)
downsample_to_origin=cv2.resize(downsample,(w-1,h-1),interpolation=cv2.INTER_CUBIC)
cv2.imwrite('/home/guozy/BISHE/SR_Collection/result/'+ subpath[choice]+'/downsample_to_origin_by_Bicubic.jpg',downsample_to_origin,[int(cv2.IMWRITE_JPEG_QUALITY), 100])

data = (ToTensor()(downsample))
data = data.to(device)
out = model(data)
out = out.permute(1,2,0).cpu()
out = out.detach().numpy()
out*= 255.0
out = out.clip(0, 255).astype(np.int8)
result = Image.fromarray(out, mode="RGB")
result.save('/home/guozy/BISHE/SR_Collection/result/'+ subpath[choice]+'/downsample_to_origin_by_NN.jpg')

p1 = psnr(downsample_to_origin, img)
s1 = ssim(downsample_to_origin, img, channel_axis=2)  
m1 = mse(downsample_to_origin, img)
print('BiCubic下采样超分辨重建结果： psnr:{} , ssim:{}, mse:{}\n'.format(p1,s1,m1))

img = img.astype(np.int8)
p2 = psnr(out, img)
s2 = ssim(out, img, channel_axis=2)
m2= mse(out, img)
print('NN下采样超分辨重建结果： psnr:{} , ssim:{}, mse:{}\n'.format(p2,s2,m2))

BiCubic下采样超分辨重建结果： psnr:33.58351656302146 , ssim:0.9418488522265225, mse:28.49237847222222

NN下采样超分辨重建结果： psnr:13.867772361019917 , ssim:0.31628098076666417, mse:2668.720703125

